In [2]:
from utils import read_xml_content
from pathlib import Path
%config Completer.use_jedi = False


In [13]:
def area(bb): 
    x1,y1,x2,y2 = bb
    return (x2-x1)*(y2-y1)

def IoU(bb, BB):
    x1, y1, x2, y2 = bb
    X1, Y1, X2, Y2 = BB
    x1_Int = max(x1, X1)
    x2_Int = min(x2, X2)
    y1_Int = max(y1, Y1)
    y2_Int = min(y2, Y2)
    AInt = area((x1_Int, y1_Int, x2_Int,  y2_Int))
    return AInt/(area(bb) + area(BB) - AInt)

def nms(anns, nms_iou_th=0.8):
    anns = sorted(anns, key=lambda x: x['score'])
    i = 0 
    while i < len(anns):
        bb1 = anns[i]['bbox']
        j = i + 1
        while j < len(anns):
            bb2 = anns[j]['bbox']
            if IoU(bb1, bb2) > nms_iou_th: anns.pop(j)
            else: j += 1
        i += 1
    return anns

def get_metrics(gt_bbs, dt_bbs, IoU_th, with_nms=False, nms_th=0.8):
    used_dts = []
    dt_bbs = nms(dt_bbs, nms_th) if with_nms else dt_bbs
    for gt_bb in gt_bbs:
        for dt_id, dt_bb in enumerate(dt_bbs):
            if IoU(gt_bb, dt_bb) > IoU_th and dt_id not in used_dts:
                used_dts.append(dt_id)
                break
                
    TP = len(used_dts)
    FP = len(dt_bbs) - TP
    FN = len(gt_bbs) - TP

    p = TP/(TP + FP) if TP + FP > 0 else 0
    r = TP/(TP + FN) if TP + FP > 0 else 0
    return p, r

In [14]:
gts_path = Path('/mnt/data/smarttray/dataset_05jul2021_av3b')
dts_path = Path('/mnt/data/smarttray/dataset_05jul2021_av3b/two_detector_results')

avg_r, avg_p, i = 0, 0, 0
for ds_path in dts_path.iterdir():
    for dt_ann_path in ds_path.iterdir():
        gt_ann_path = gts_path/ds_path.stem/'val/annotations'/dt_ann_path.name
        
        dt_anns = read_xml_content(dt_ann_path)
        gt_anns = read_xml_content(gt_ann_path)
        
        dt_bbs = [[x1,y1,x2,y2] for _,x1,y1,x2,y2 in dt_anns]
        gt_bbs = [[x1,y1,x2,y2] for _,x1,y1,x2,y2 in gt_anns]
              
        
        p,r = get_metrics(dt_bbs, gt_bbs, IoU_th=0.8)
        avg_p += p; avg_r += r
        i += 1
        
avg_p/i, avg_r/i

(0.8546251109139298, 0.6912306106017279)